In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob
from src.utils import Data, markdown_html
import matplotlib.pyplot as plt
from bokeh.io import show, save, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, BasicTicker)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.palettes import Inferno256
from bokeh.plotting import figure

In [ ]:
maryland = Data(state='MD')

In [ ]:
data = maryland.geo \
    .merge(maryland.zip_map, on ='Zip', how = 'right') \
    .merge(maryland.zip_covid, on ='Zip', how ='left')\
    .merge(maryland.zip_population.assign(Zip = lambda d: d.Zip.astype(int)), on ='Zip')\
    .assign(Date = lambda d: d.Date.fillna(d.Date.max()))\
    .pipe(lambda d: d[d.Date==d.Date.max()])  \
    .filter(['Zip','City','State','Cases','Date','geometry', 'Population']) \
    .assign(Cases = lambda d: d.Cases.fillna(0)) \
    .pipe(lambda d: d[~pd.isnull(d.geometry)]) \
    .assign(per_population = lambda d: d.Cases / d.Population.astype(int) * 1000)
data.head()

In [ ]:
geosource = GeoJSONDataSource(geojson = data.drop('Date', axis=1).to_json())
color_mapper = LinearColorMapper(palette=Inferno256, 
                           low=data.per_population.min(), 
                           high=data.per_population.max())

p = figure(title = 'COVID19 in Maryland (Cases per 1,000 people)', 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.title.text_font_size = '15pt'

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field':'per_population',
                                'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p.add_tools(HoverTool(tooltips = [('City','@City'),
                                  ('Zip code','@Zip'),
                                  ('Population','@Population'),
                                  ('Cases', '@Cases'),
                                  ('Cases/1k population', '@per_population')]))

#color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                    orientation='horizontal')
p.add_layout(color_bar, 'below')
output_file('output.html')
save(p)


In [ ]:
markdown_html('output.html','COVID.html')

In [ ]:
data = maryland.geo \
    .merge(maryland.zip_map, on ='Zip', how = 'right') \
    .merge(maryland.zip_covid, on ='Zip', how ='left')\
    .merge(maryland.zip_population.assign(Zip = lambda d: d.Zip.astype(int)), on ='Zip')\
    .assign(Date = lambda d: d.Date.fillna(d.Date.max()))
data.head()

In [ ]:
ts_data = maryland\
    .zip_covid\
    .pipe(pd.pivot_table, columns = 'Zip', 
          values='Cases', index='Date')\
    .fillna(method='ffill', axis=0) \
    .fillna(0)
ts_data.head()

In [ ]:
source = ColumnDataSource(maryland.zip_covid.query('Zip == 21215'))
p = figure(x_axis_type="datetime", tools='hover,save,pan,box_zoom,reset,wheel_zoom')
p.line(x='Date', y='Cases', source=source)
show(p)